In [16]:
import sympy
from sympy import *
from utils import *
from gates import *
from qbits import *

In [2]:
x = gate(1, X)
y = gate(1, Y)
cnot = gate(2, CNOT)

In [3]:
xx = gate_call(x, [2])
yy = gate_call(y, [0])
ccnot = gate_call(cnot, [1,2])

In [8]:
make_U([yy, xx], 3).mat

⎡0  0   0  0   0  -ⅈ  0  0 ⎤
⎢                          ⎥
⎢0  0   0  0   ⅈ  0   0  0 ⎥
⎢                          ⎥
⎢0  0   0  0   0  0   0  -ⅈ⎥
⎢                          ⎥
⎢0  0   0  0   0  0   ⅈ  0 ⎥
⎢                          ⎥
⎢0  -ⅈ  0  0   0  0   0  0 ⎥
⎢                          ⎥
⎢ⅈ  0   0  0   0  0   0  0 ⎥
⎢                          ⎥
⎢0  0   0  -ⅈ  0  0   0  0 ⎥
⎢                          ⎥
⎣0  0   ⅈ  0   0  0   0  0 ⎦


Matrix([
[0,  0, 0,  0, 0, -I, 0,  0],
[0,  0, 0,  0, I,  0, 0,  0],
[0,  0, 0,  0, 0,  0, 0, -I],
[0,  0, 0,  0, 0,  0, I,  0],
[0, -I, 0,  0, 0,  0, 0,  0],
[I,  0, 0,  0, 0,  0, 0,  0],
[0,  0, 0, -I, 0,  0, 0,  0],
[0,  0, I,  0, 0,  0, 0,  0]])